# Multiple Results


In this notebook we show how you can in a single request, have the LLM model return multiple results per prompt. This is useful for running experiments where you want to evaluate the robustness of your prompt and the parameters of your config against a particular large language model.


In [ ]:
!python -m pip install semantic-kernel==0.5.1.dev0

In [ ]:
from services import Service

# Select a service to use for this notebook (available services: OpenAI, AzureOpenAI, HuggingFace)
selectedService = Service.AzureOpenAI

In [ ]:
import semantic_kernel as sk

if selectedService == Service.OpenAI or selectedService == Service.AzureOpenAI:
    from semantic_kernel.connectors.ai.open_ai.prompt_execution_settings.open_ai_prompt_execution_settings import (
        OpenAITextPromptExecutionSettings,
        OpenAIChatPromptExecutionSettings,
    )
    from semantic_kernel.connectors.ai.open_ai.prompt_execution_settings.azure_chat_prompt_execution_settings import (
        AzureChatPromptExecutionSettings,
    )
    from semantic_kernel.connectors.ai.open_ai import (
        AzureTextCompletion,
        AzureChatCompletion,
        OpenAITextCompletion,
        OpenAIChatCompletion,
    )
if selectedService == Service.HuggingFace:
    from semantic_kernel.connectors.ai.hugging_face import HuggingFaceTextCompletion

First, we will set up the text and chat services we will be submitting prompts to.


In [ ]:
kernel = sk.Kernel()

# Configure Azure LLM service
if selectedService == Service.AzureOpenAI:
    deployment, api_key, endpoint = sk.azure_openai_settings_from_dot_env()
    azure_text_service = AzureTextCompletion(
        deployment_name="gpt-35-turbo-instruct", endpoint=endpoint, api_key=api_key
    )  # set the deployment name to the value of your text model (e.g. gpt-35-turbo-instruct or text-davinci-003)
    azure_chat_service = AzureChatCompletion(
        deployment_name="gpt-35-turbo", endpoint=endpoint, api_key=api_key
    )  # set the deployment name to the value of your chat model

# Configure OpenAI service
if selectedService == Service.OpenAI:
    api_key, org_id = sk.openai_settings_from_dot_env()
    oai_text_service = OpenAITextCompletion(ai_model_id="gpt-3.5-turbo-instruct", api_key=api_key, org_id=org_id)
    oai_chat_service = OpenAIChatCompletion(ai_model_id="gpt-3.5-turbo", api_key=api_key, org_id=org_id)

# Configure Hugging Face service
if selectedService == Service.HuggingFace:
    hf_text_service = HuggingFaceTextCompletion(ai_model_id="distilgpt2", task="text-generation")

Next, we'll set up the completion request settings for text completion services.


In [ ]:
oai_text_prompt_execution_settings = OpenAITextPromptExecutionSettings(
    extension_data={
        "max_tokens": 80,
        "temperature": 0.7,
        "top_p": 1,
        "frequency_penalty": 0.5,
        "presence_penalty": 0.5,
        "number_of_responses": 3,
    }
)

## Multiple Open AI Text Completions


In [ ]:
if selectedService == Service.OpenAI:
    prompt = "what is the purpose of a rubber duck?"
    results = await oai_text_service.complete(prompt=prompt, settings=oai_text_prompt_execution_settings)
    i = 1
    for result in results:
        print(f"Result {i}: {result}")
        i += 1

## Multiple Azure Open AI Text Completions


In [ ]:
if selectedService == Service.AzureOpenAI:
    prompt = "provide me a list of possible meanings for the acronym 'ORLD'"
    results = await azure_text_service.complete(prompt=prompt, settings=oai_text_prompt_execution_settings)
    i = 1
    for result in results:
        print(f"Result {i}: {result}")
        i += 1

## Multiple Hugging Face Text Completions


In [ ]:
if selectedService == Service.HuggingFace:
    from semantic_kernel.connectors.ai.hugging_face.hf_prompt_execution_settings import (
        HuggingFacePromptExecutionSettings,
    )

    hf_prompt_execution_settings = HuggingFacePromptExecutionSettings(
        extension_data={"max_new_tokens": 80, "temperature": 0.7, "top_p": 1}
    )

In [ ]:
if selectedService == Service.HuggingFace:
    prompt = "The purpose of a rubber duck is"
    results = await hf_text_service.complete(prompt=prompt, prompt_execution_settings=hf_prompt_execution_settings)
    print("".join(results))

Here, we're setting up the settings for Chat completions.


In [ ]:
oai_chat_prompt_execution_settings = OpenAIChatPromptExecutionSettings(
    max_tokens=80,
    temperature=0.7,
    top_p=1,
    frequency_penalty=0.5,
    presence_penalty=0.5,
    number_of_responses=3,
)

## Multiple OpenAI Chat Completions


In [ ]:
if selectedService == Service.OpenAI:
    role = "user"
    content = (
        "It's a beautiful day outside, birds are singing, flowers are blooming. On days like these, kids like you..."
    )
    message = {"role": role, "content": content}
    results = await oai_chat_service.complete_chat(messages=[message], settings=oai_chat_prompt_execution_settings)
    i = 0
    for result in results:
        print(f"Result {i}: {result[0]}")
        i += 1

## Multiple Azure OpenAI Chat Completions


In [ ]:
az_oai_prompt_execution_settings = AzureChatPromptExecutionSettings(
    max_tokens=80,
    temperature=0.7,
    top_p=1,
    frequency_penalty=0.5,
    presence_penalty=0.5,
    number_of_responses=3,
)

In [ ]:
if selectedService == Service.AzureOpenAI:
    role = "user"
    content = "Tomorow is going to be a great day, I can feel it. I'm going to wake up early, go for a run, and then..."
    message = {"role": role, "content": content}
    results = await azure_chat_service.complete_chat(messages=[message], settings=az_oai_prompt_execution_settings)
    i = 0
    for result in results:
        print(f"Result {i}: {result[0]}")
        i += 1

## Streaming Multiple Results

Here is an example pattern if you want to stream your multiple results. Note that this is not supported for Hugging Face text completions at this time.


In [ ]:
if selectedService == Service.OpenAI:
    import os
    from IPython.display import clear_output
    import time

    # Determine the clear command based on OS
    clear_command = "cls" if os.name == "nt" else "clear"

    prompt = "what is the purpose of a rubber duck?"
    stream = oai_text_service.complete_stream(prompt=prompt, settings=oai_text_prompt_execution_settings)
    number_of_responses = oai_text_prompt_execution_settings.number_of_responses
    texts = [""] * number_of_responses

    last_clear_time = time.time()
    clear_interval = 0.5  # seconds

    # Note: there are some quirks with displaying the output, which sometimes flashes and disappears.
    # This could be influenced by a few factors specific to Jupyter notebooks and asynchronous processing.
    # The following code attempts to buffer the results to avoid the output flashing on/off the screen.

    async for results in stream:
        current_time = time.time()

        # Update texts with new results
        for idx, result in enumerate(results):
            if idx < number_of_responses:
                texts[idx] += result

        # Clear and display output at intervals
        if current_time - last_clear_time > clear_interval:
            clear_output(wait=True)
            for idx, text in enumerate(texts):
                print(f"Result {idx + 1}: {text}")
            last_clear_time = current_time

    print("----------------------------------------")